In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
path = '/content/gdrive/MyDrive/Colab Notebooks/Santi/'

Mounted at /content/gdrive/


In [64]:
import numpy as np
import pandas as pd
from pandas import read_csv
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

datasetOri = read_csv(path+"depression.csv")
#print(datasetOri.describe())
#ada yg aneh di column 'VisualProblem', string value
datasetOri.drop('VisualProblem', inplace=True, axis=1)
datasetOri.head()


,VAR00001,Birthday,Age,Gender,Race,Religion,EduLevel,MaritalStatus,NumberOfChildren,LivingStatus,EmployedNow,Jobnow,EmployeBefore,PastJob,Income,IncomeMonthlyCost,VisitCHCTmsYears,LeisureActivity,AttendLAtimesMonth,Comorbidity,HealthCondition,HearingProblem,HearingAids,WhatVisualProblem,PermanentTeeth,OralStatus,Medication,ADLSBarthelIndex,IADLSLawton,CogFunctionSPMSQ,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,RL10,RL15,RL16,RL19,RL20,TTUCLA,RD1,RD2,RD3,RD4,RD5,RD6,RD7,RD8,RD9,RD10,RD11,RD12,RD13,RD14,RD15,TTGDS,RG,RR,RL,RE,RM,RLv,REn,REB,RICom,RHP,RVP,RPT,NewRPT,RHP2,HP3,Rcomor,CatGDS,CatGDS2
0,1,195908,60,1,2,1,4,2,8,3,0,0,1,2,400000,2,0,0,0,1,4,0,0,0,1,3,0,95,6,10,5,5,5,5,6,6,6,4,4,3,...,2,3,3,2,2,48,0,1,1,1,0,1,0,1,1,1,0,1,0,1,1,10,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,1,1,1
1,2,195803,61,1,2,1,4,4,0,4,1,2,0,0,400000,2,12,0,0,1,4,0,0,0,1,4,0,95,8,10,5,5,6,6,6,5,4,4,4,4,...,3,4,2,2,2,48,0,1,1,1,1,0,1,0,1,1,0,1,0,1,0,9,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,1,1
2,3,195807,61,1,2,1,4,2,3,3,0,0,0,0,400000,2,0,0,0,0,3,0,0,1,1,4,0,95,7,4,5,5,5,4,4,4,4,5,5,5,...,2,1,2,3,3,54,0,1,1,1,0,1,1,0,0,0,0,1,0,1,1,8,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,1
3,4,195302,66,1,1,1,3,2,3,3,0,0,1,2,500000,2,9,0,0,1,4,0,0,1,1,3,1,85,0,10,7,7,7,6,5,5,6,6,6,6,...,2,3,3,3,3,56,0,1,1,1,0,0,1,1,1,0,0,1,0,1,1,9,0,0,0,0,0,0,1,1,0,0,1,1,1,0,0,1,1,1
4,5,195704,62,2,3,1,1,2,2,3,0,0,0,0,500000,2,8,0,0,1,3,0,0,0,1,3,0,95,8,4,5,5,5,5,4,4,6,6,6,6,...,1,1,2,1,1,47,0,1,1,1,0,1,1,0,1,1,0,1,0,1,0,9,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,1,1


In [65]:
x = datasetOri.iloc[:,2:29]
y = datasetOri.iloc[:,29]
x_train, x_eval,y_train,y_eval = train_test_split(x,y,test_size=0.2,random_state=0)

scaler_model = MinMaxScaler()
x_scaled = scaler_model.fit(x_train)
x_train=pd.DataFrame(scaler_model.transform(x_train),columns=x_train.columns,index=x_train.index)

scaler_model.fit(x_eval)
x_eval=pd.DataFrame(scaler_model.transform(x_eval),columns=x_eval.columns,index=x_eval.index)

In [66]:
#Creating Feature Columns
feat_cols=[]
for cols in x.columns:
    column=tf.feature_column.numeric_column(cols)
    feat_cols.append(column)
    
print(feat_cols)

[NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Gender', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Race', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Religion', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='EduLevel', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='MaritalStatus', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='NumberOfChildren', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='LivingStatus', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='EmployedNow', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Jobnow', shape=(1,), default_value=None, dtype=tf.float32, nor

In [67]:
#The estimator model
model=tf.estimator.DNNRegressor(hidden_units=[6,10,6],feature_columns=feat_cols)

#the input function
input_func=tf.compat.v1.estimator.inputs.pandas_input_fn(x_train,y_train,batch_size=10,num_epochs=1000,shuffle=True)

#Training the model
model.train(input_fn=input_func,steps=1000)

#Evaluating the model
train_metrics=model.evaluate(input_fn=input_func,steps=1000)

#Now to predict values we do the following
pred_input_func=tf.compat.v1.estimator.inputs.pandas_input_fn(x=x_eval,y=y_eval,batch_size=10,num_epochs=1,shuffle=False)
preds=model.predict(input_fn=pred_input_func)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpr3klemp2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:ten

<generator object Estimator.predict at 0x7f26fc6bee08>

In [68]:
predictions=list(preds)
final_pred=[]
for pred in predictions:
    final_pred.append(pred["predictions"])
    
test_metric=model.evaluate(input_fn=pred_input_func,steps=1000) 

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpr3klemp2/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-03T08:09:19Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpr3klemp2/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.35502s
INFO:tensorflow:Finished evaluation at 2021-01-03-08:09:19
INFO:tensorflow:Saving dict for global step 1000: average_loss = 44.087547, global_step = 1000, label/mean = 8.093863, loss = 44.1234, prediction/mean = 1.8377213
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmpr3klemp2/model.ckpt-1000
